# Question Answering

Question Answering은 context를 읽고 question애 올바르게 답하는(answering) task입니다.
크게 2가지 형태가 존재합니다.
* extractive: 주어진 context에서 답(answer)을 찾는 것
* abstractive: 주어진 context로 부터 답(answer)을 생성해내는 것

KLUE의 MRC 데이터셋을 이용하여 Question Answering을 수행하겠습니다.

## 1. Model, Tokenizer 다운로드

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 24.1 MB/s 
     |████████████████████████████████| 7.6 MB 61.8 MB/s 
     |████████████████████████████████| 182 kB 77.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 19.1 MB/s 
     |████████████████████████████████| 212 kB 53.8 MB/s 
     |████████████████████████████████| 132 kB 58.9 MB/s 
     |████████████████████████████████| 127 kB 54.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


QA Task에서 사용할 BERT 모델은 ['beomi/kcbert-base'](https://huggingface.co/beomi/kcbert-base)입니다.

In [2]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained('beomi/kcbert-base')
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

## 2. Dataset 다운로드

사용할 데이터셋은 KLUE-benchmark의 MRC 데이터셋입니다.

* https://github.com/KLUE-benchmark/KLUE
* https://huggingface.co/datasets/klue/viewer/mrc/train

In [3]:
from datasets import load_dataset

data = load_dataset('klue','mrc')

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})


데이터셋이 어떤 구조로 이루어져있는지 확인합니다.

In [5]:
print(data['train'][0])

{'title': '제주도 장마 시작 … 중부는 이달 말부터', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'news_category': '종합', 'source': 'hankyung', 'guid': 'klue-mrc-v1_train_12759', 'is_impossible': False, 'question_type': 1, 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': {'answer_start': [478, 478], 'text'

tokenizer가 정상적으로 동작하는 확인합니다.

In [6]:
print(tokenizer.tokenize(data['train']['question'][0]))

['북', '##태', '##평', '##양', '기', '##단', '##과', '오호', '##츠', '##크', '##해', '기', '##단이', '만나', '국내에', '머', '##무', '##르는', '기간', '##은', '?']


## 3. Preprocessing

데이터 중에 잘못된 데이터가 있는 지 확인하고 answer_end를 추가합니다.

In [7]:
# 데이터 중에 answer_start index에 위치한 글자와 text의 첫번째 글자가 일치하지 않은 경우가 있다. → 잘못된 데이터의 존재

def filtering(contexts, questions, answers):
    valid_contexts = []
    valid_questions = []
    answer_starts = []
    answer_ends = []
    for context, question, answer in zip(contexts, questions ,answers):
        answer_start = answer['answer_start'][0]
        answer_text = answer['text'][0]
        answer_end = answer_start + len(answer_text)

        if context[answer_start:answer_end] == answer_text:
            valid_contexts.append(context)
            valid_questions.append(question)
            answer_starts.append(answer_start)
            answer_ends.append(answer_end-1)
    
    assert len(answer_starts) == len(valid_contexts) and len(valid_contexts) == len(valid_questions), "context 갯수와 answer의 갯수가 일치하지 않습니다."

    return {'contexts':valid_contexts, 'questions':valid_questions, 'answer_starts':answer_starts, 'answer_ends':answer_ends}

In [8]:
from tqdm.notebook import tqdm

def preprocess(contexts, questions, answers):
    datasets = filtering(contexts, questions, answers)
    answer_starts = datasets.pop('answer_starts')
    answer_ends = datasets.pop('answer_ends')

    inputs = tokenizer(
        datasets['questions'],
        datasets['contexts'],
        truncation='only_second',
        padding=True,
        return_offsets_mapping=True
    )

    offset_mapping = inputs.pop('offset_mapping')
    start_positions = []
    end_positions = []

    for i, offset in tqdm(enumerate(offset_mapping), total=len(offset_mapping), desc='preprocessing'):
        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 2

        if offset[context_start][0] > answer_ends[i] or offset[context_end][1] < answer_starts[i]:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_pos, end_pos = -1, -1
            for idx, (start, end) in enumerate(offset):
                if idx >= context_start and start <= answer_starts[i] <= end:
                    start_pos = idx
                if idx >= context_start and start <= answer_ends[i] <= end:
                    end_pos = idx

            # 정답이 context에 온전히 포함되지 않고 잘리는 경우가 존재한다.
            if end_pos == -1:
                end_pos = len(offset) - 2

            start_positions.append(start_pos)
            end_positions.append(end_pos)
    
    assert len(start_positions) == len(inputs['input_ids'])

    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions

    return inputs

In [9]:
train_data = preprocess(data['train']['context'], data['train']['question'], data['train']['answers'])
test_data = preprocess(data['validation']['context'], data['validation']['question'], data['validation']['answers'])

preprocessing:   0%|          | 0/16241 [00:00<?, ?it/s]

preprocessing:   0%|          | 0/5432 [00:00<?, ?it/s]

전처리한 데이터를 Pytorch 형태의 Dataset으로 만들어줍니다.

In [10]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self,data):
        self.data = data

    def __getitem__(self, idx):
        return {k:torch.tensor(v[idx]) for k, v in self.data.items()}

    def __len__(self):
        return len(self.data['start_positions'])

In [11]:
train_dataset = QADataset(train_data)
test_dataset = QADataset(test_data)

In [12]:
train_dataset[0]

{'input_ids': tensor([    2,  1611,  4048,  4492,  4237,   414,  4281,  4128, 25740,  4838,
          4147,  4032,   414, 11413, 10448, 20389,  1337,  4211,  8941, 15825,
          4057,    32,     3,  2303,  4327,  4248,  2492, 19603, 11572,  4046,
         11527,  7971,  8435, 11337,    17,  8270,   963,  2635, 13354, 25883,
          2289,  4482,  8043, 18962,  4808,  8384, 24075,  2451,  4182,  1300,
          4051,  2492, 19603,  8435,  4339,  2525, 14060,  4020,    17, 11572,
          4046, 28024,  4113, 24183, 11527, 22824,  1340, 17180,  4113,  8032,
          2492,  4168,  4203,  4124,  4042, 10741,  7965,  2451,  4272, 11527,
          1789,  4045,  1476,   609,  4706, 25334,  3443,  4216,  8055, 16601,
          8524,  8726,  4072, 22396,     1,  2408,  4296,  7966,  8298, 28206,
         25659,  4020,    17, 14076,  4042,  2492, 19432,  3288,  4482,  8043,
            21,    95, 16423,    15,  9403,  8892,  7995,  8525, 14981,  8435,
         11337,    17,  2492, 19432,   

## 4. Training

Transformers에서 제공하는 Training용 함수인 DefaultDataCollator, TrainingArguments, Trainer를 사용해서 학습합니다.

In [13]:
from transformers import DefaultDataCollator, TrainingArguments, Trainer

data_collator = DefaultDataCollator()

training args에 configuration정보를 저장합니다.

In [14]:
training_args = TrainingArguments(
    output_dir = './outputs',
    logging_dir = './logs',
    num_train_epochs = 5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=1
)

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda', index=0)

Trainer 함수를 사용하여 학습에 사용할 trainer를 제작합니다.

In [16]:
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

In [17]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16241
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5080
  Number of trainable parameters = 108329474


Step,Training Loss
100,3.505500
200,3.216900
300,3.082700
400,2.732700
500,2.502100
600,2.330500
700,2.151600
800,2.136600
900,2.099300
1000,2.106700


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [outputs/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [outputs/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to ./outputs/checkpoint-2000
Configuration saved in ./outputs/checkpoint-2000/config.json
Model weights saved in ./outputs/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [outputs/checkpoint-1500] due to args.save_total_limit
Saving model checkpoint to ./outputs/

TrainOutput(global_step=5080, training_loss=1.2293975229338399, metrics={'train_runtime': 4471.0369, 'train_samples_per_second': 18.162, 'train_steps_per_second': 1.136, 'total_flos': 1.2432775271922e+16, 'train_loss': 1.2293975229338399, 'epoch': 5.0})

In [18]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5432
  Batch size = 16


{'eval_loss': 2.701833963394165,
 'eval_runtime': 104.878,
 'eval_samples_per_second': 51.794,
 'eval_steps_per_second': 3.242,
 'epoch': 5.0}

In [20]:
from transformers import pipeline

nlp = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

In [22]:
context = r"""
기상청에 따르면, 이날 새벽부터 낮 사이 서울과 인천·경기·강원 내륙 및 산지에는 눈이 내리겠다. 수도권과 강원 산지는 눈이 쌓이는 곳도 있겠다. 경기 북부 일부 지역은 새벽부터 아침 사이 많은 눈이 내리면서 대설특보가 발표될 가능성도 있다.
"""

print(nlp(question="눈이 내리는 지역은?", context=context))
print(nlp(question="대설특보가 발표되는 지역은?", context=context))

{'score': 8.76534969281896e-14, 'start': 83, 'end': 88, 'answer': '경기 북부'}
{'score': 7.020170555982475e-11, 'start': 83, 'end': 95, 'answer': '경기 북부 일부 지역은'}


In [30]:
context = r"""
일본이 '무적함대' 스페인에도 역전승을 거두며 조 1위로 2022 카타르 월드컵 16강에 진출했다. 스페인이 2위로 16강에 오른 가운데 독일은 코스타리카에 재역전승을 거뒀으나 3위에 그쳐 두 대회 연속 조별리그에서 탈락했다. 일본은 2일(한국시간) 카타르 알라이얀의 칼리파 인터내셔널 스타디움에서 열린 카타르 월드컵 조별리그 E조 최종 3차전에서 전반 스페인의 알바로 모라타에게 선제골을 허용했으나 후반 연속 골에 힘입어 2-1로 이겼다.
"""

print(nlp(question="일본의 조 순위는?", context=context))
print(nlp(question="카타르 월드컵 16강에 진출한 나라는?", context=context))
print(nlp(question="조 1위로 2022 카타르 월드컵 16강에 진출한 나라는?", context=context))
print(nlp(question="독일이 재역전승을 거둔 나라는?", context=context))

{'score': 7.983120241250585e-12, 'start': 6, 'end': 32, 'answer': "무적함대' 스페인에도 역전승을 거두며 조 1위로"}
{'score': 0.012238853611052036, 'start': 1, 'end': 4, 'answer': '일본이'}
{'score': 0.03482533618807793, 'start': 1, 'end': 4, 'answer': '일본이'}
{'score': 0.005576292984187603, 'start': 1, 'end': 4, 'answer': '일본이'}


In [37]:
context = r"""
이순신(1545년 4월 28일 (음력 3월 8일) ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다. 본관은 덕수, 자는 여해, 시호는 충무였으며, 한성 출신이었다. 문반 가문 출신으로 1576년 무과에 급제하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라좌도수사를 거쳐 정헌대부 삼도수군통제사에 이르렀다.
"""

print(nlp(question="이순신의 출신은?", context=context))
print(nlp(question="이순신이 무과에 급제한 연도는?", context=context))
print(nlp(question="이순신의 급제한 직책은?", context=context))

{'score': 7.033596193650737e-05, 'start': 110, 'end': 112, 'answer': '문반'}
{'score': 0.07732417434453964, 'start': 121, 'end': 126, 'answer': '1576년'}
{'score': 2.630553197491281e-08, 'start': 127, 'end': 130, 'answer': '무과에'}
